In [1]:
# 导入数据集
from Toolkit import *
import sys  
sys.path.append('/root/zqh/filter_test')
from FingerPrint_5_quick import *
from United_model import *

Pathlist = [
    '/root/zqh/NewDataSet/BCG_ZQH2.pt',
    '/root/zqh/NewDataSet/BCG_QJF1.pt',
    '/root/zqh/NewDataSet/BCG_WCM1.pt',
    '/root/zqh/NewDataSet/New_data_pa1.pt',
    '/root/zqh/NewDataSet/New_data_pa2.pt',
    '/root/zqh/NewDataSet/New_data_pa3.pt',
    '/root/zqh/NewDataSet/New_data_pa5.pt',
    '/root/zqh/NewDataSet/New_data_pa6.pt',
    '/root/zqh/NewDataSet/New_data_pa7.pt',
    '/root/zqh/NewDataSet/New_data_pa8.pt',
    '/root/zqh/NewDataSet/New_data_pa9.pt',
    '/root/zqh/NewDataSet/New_data_pa10.pt',
    '/root/zqh/NewDataSet/New_data_pa11.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa12.pt',
    '/root/zqh/NewDataSet/New_data_pa13.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa14.pt',
    '/root/zqh/NewDataSet/New_data_pa15.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa16.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa17.pt',
    '/root/zqh/NewDataSet/New_data_pa18.pt',
    '/root/zqh/NewDataSet/New_data_pa19.pt',
    '/root/zqh/NewDataSet/New_data_pa20.pt',
    '/root/zqh/NewDataSet/New_data_pa21.pt',
    '/root/zqh/NewDataSet/New_data_pa22.pt',
    '/root/zqh/NewDataSet/New_data_pa23.pt',
    '/root/zqh/NewDataSet/New_data_pa24.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa25.pt',
    '/root/zqh/NewDataSet/New_data_pa26.pt',
    '/root/zqh/NewDataSet/New_data_pa27.pt',
    '/root/zqh/NewDataSet/New_data_pa28.pt',
    '/root/zqh/NewDataSet/New_data_pa29.pt',
    '/root/zqh/NewDataSet/New_data_pa30.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa31.pt',
    '/root/zqh/NewDataSet/New_data_pa32.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa33.pt',
    '/root/zqh/NewDataSet/New_data_pa34.pt',
    '/root/zqh/NewDataSet/New_data_pa35.pt',
    '/root/zqh/NewDataSet/New_data_pa36.pt',
    '/root/zqh/NewDataSet/New_data_pa37.pt',
    '/root/zqh/NewDataSet/New_data_pa38.pt',
    '/root/zqh/NewDataSet/New_data_pa39.pt',
    '/root/zqh/NewDataSet/New_data_pa40.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_caoan615.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_dj613.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_dxt613.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_ltm613.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_qjf612.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_rrx613.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_sample1.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_sjj612.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_tt612.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_wcm612.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_wg613.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_whd612.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_wxy.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_zj612.pt',
    # '/root/zqh/BCGDataSet/modify_extract_Single_resolution_zqh1.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_zzp612.pt',
]
oneperson_begin = 0
oneperson_end = 30
oneperson_nums = oneperson_end - oneperson_begin
persons = len(Pathlist)
data = get_ResUnet_data(Pathlist=Pathlist, oneperson_begin=oneperson_begin, oneperson_end=oneperson_end)[:,:,:].detach()
label = torch.zeros(oneperson_nums*persons)
for i in range(persons):
    label[i*oneperson_nums:(i+1)*oneperson_nums] = i

def splitDataSet(data, label, persons, oneperson_nums): # 要求输入 shape: N x
    X_train = data[0:20,:]
    X_test = data[20:30,:]
    y_train = label[0:20]
    y_test = label[20:30]
    for i in range(1,persons):
        X_train = torch.cat([X_train, data[oneperson_nums*i:oneperson_nums*i+20,:]],dim=0)
        X_test = torch.cat([X_test, data[oneperson_nums*i+20:oneperson_nums*i+30,:]],dim=0)
        y_train = torch.cat([y_train, label[oneperson_nums*i:oneperson_nums*i+20]],dim=0)
        y_test = torch.cat([y_test, label[oneperson_nums*i+20:oneperson_nums*i+30]],dim=0)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = splitDataSet(data.cpu().squeeze(1), label, persons, oneperson_nums)
# 导入数据集

# 随机森林
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, random_state=42)
model_rf.fit(X_train, y_train)

# KNN
from sklearn.neighbors import KNeighborsClassifier
def model_knn(persons, X_train, X_test, y_train, y_test, k):
    ans_list = []
    for i in range(X_test.shape[0]):
        tmp = X_test[i].repeat(X_train.shape[0], 1)
        distancelist = torch.sum((tmp-X_train)**2,dim=1)
        distancelist = torch.cat([distancelist.unsqueeze(1), y_train.unsqueeze(1)],dim=1)
        distancelist = distancelist.tolist()
        distancelist = sorted(distancelist, key = lambda x:(x[0]))
        distancelist = np.array(distancelist)
        distancelist = distancelist[:k,1:2]
        record = torch.zeros(persons)
        for j in range(k):
            record[int(distancelist[j][0])] += 1
        maxvalue,maxindex = torch.max(record,dim=0)
        ans_list.append(int(maxindex))
    return ans_list

# SVM
from sklearn import svm
model_svm = svm.SVC(kernel = 'linear')
model_svm.fit(X_train, y_train)

# XGboost
import xgboost as xgb
model_xgb = xgb.XGBClassifier()
model_xgb.fit(X_train, y_train)

ans1 = model_rf.predict(X_test)
ans2 = model_knn(persons=len(Pathlist), X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, k=5)
ans3 = model_svm.predict(X_test)
ans4 = model_xgb.predict(X_test)
# print(ans1,ans2,ans3,ans4)

# 计算集成学习最终准确率
right = 0
persons=len(Pathlist)
final_list = []
for i in range(X_test.shape[0]):
    record = torch.zeros(persons)
    record[int(ans1[i])]+=1
    record[int(ans2[i])]+=1
    record[int(ans3[i])]+=1
    record[int(ans4[i])]+=1
    maxvalue,maxindex = torch.max(record,dim=0)
    final_list.append(int(maxindex))
    if int(maxindex) == y_test[i]:
        right+=1
print(right/X_test.shape[0])
print(final_list)



0.9508771929824561
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 48, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 33, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 7, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 19, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 9, 14, 17, 16, 14, 14, 16, 17, 17, 17, 17, 17, 17, 6, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 8, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 48, 24, 24, 24, 24, 24, 24, 24, 24, 24, 40, 25, 25, 25, 6, 14, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27,